# Part A: Leaf Node and In-place Operation

In [ ]:
import torch

w = torch.tensor([2.0, 3.0], requires_grad=True)

output = w[0] * w[1]

try:
    w += 1.0  
except RuntimeError as e:
    print(e)

output.backward()

print(f"grad: {w.grad}")
print(f"w: {w}")

a leaf Variable that requires grad is being used in an in-place operation.
grad tensor([3., 2.])
w tensor([2., 3.], requires_grad=True)


In [69]:
w = torch.tensor([2.0, 3.0], requires_grad=True)

output = w[0] * w[1]

w = w + 1.0

output.backward()

print(w.grad)

None


C:\Users\kingv\AppData\Local\Temp\ipykernel_11428\575195260.py:9: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at C:\cb\pytorch_1000000000000\work\build\aten\src\ATen/core/TensorBody.h:494.)
  print(w.grad)


In [3]:
w = torch.tensor([2.0, 3.0], requires_grad=True)

output = w[0] * w[1]

output.backward()
print("Gradient:", w.grad)  

with torch.no_grad():
    w += 1.0          

print("Updated w:", w)

Gradient: tensor([3., 2.])
Updated w: tensor([3., 4.], requires_grad=True)


# Part B: (mini-project) Training

In [56]:
import numpy as np
import pandas as pd

In [57]:
data_df = pd.read_csv("../data/BTCUSDT.csv", sep='|', nrows=10, header=None, usecols=[0,4],
                       names=['timestamp', 'close'])
data_df['timestamp'] = pd.to_datetime(data_df['timestamp'], unit='s')
data_df.set_index('timestamp', inplace=True)
print(data_df)

                       close
timestamp                   
2017-08-17 04:00:00  4261.48
2017-08-17 04:01:00  4261.48
2017-08-17 04:02:00  4280.56
2017-08-17 04:03:00  4261.48
2017-08-17 04:04:00  4261.48
2017-08-17 04:05:00  4261.48
2017-08-17 04:06:00  4261.48
2017-08-17 04:07:00  4261.48
2017-08-17 04:08:00  4261.48
2017-08-17 04:09:00  4261.48


In [58]:
data_tensor = torch.tensor(data_df.values, dtype=torch.float32)
data_tensor, data_tensor.shape

(tensor([[4261.4800],
         [4261.4800],
         [4280.5601],
         [4261.4800],
         [4261.4800],
         [4261.4800],
         [4261.4800],
         [4261.4800],
         [4261.4800],
         [4261.4800]]),
 torch.Size([10, 1]))

In [59]:
w = torch.randn_like(data_tensor, requires_grad=True)

target = torch.tensor(100)

learning_rate = 1e-9

for i in range(20):
    y = w.T @ data_tensor

    loss = (y - target) ** 2

    loss.backward()

    print(f"Epoch: {i}, loss: {loss}")

    with torch.no_grad():
        w -= learning_rate * w.grad

    w.grad.zero_()

print("weights after training : ", w)

Epoch: 0, loss: tensor([[71618736.]], grad_fn=<PowBackward0>)
Epoch: 1, loss: tensor([[29012320.]], grad_fn=<PowBackward0>)
Epoch: 2, loss: tensor([[11752701.]], grad_fn=<PowBackward0>)
Epoch: 3, loss: tensor([[4760953.5000]], grad_fn=<PowBackward0>)
Epoch: 4, loss: tensor([[1928629.2500]], grad_fn=<PowBackward0>)
Epoch: 5, loss: tensor([[781274.7500]], grad_fn=<PowBackward0>)
Epoch: 6, loss: tensor([[316489.7500]], grad_fn=<PowBackward0>)
Epoch: 7, loss: tensor([[128207.3516]], grad_fn=<PowBackward0>)
Epoch: 8, loss: tensor([[51936.3633]], grad_fn=<PowBackward0>)
Epoch: 9, loss: tensor([[21038.7383]], grad_fn=<PowBackward0>)
Epoch: 10, loss: tensor([[8522.7246]], grad_fn=<PowBackward0>)
Epoch: 11, loss: tensor([[3452.5667]], grad_fn=<PowBackward0>)
Epoch: 12, loss: tensor([[1398.5884]], grad_fn=<PowBackward0>)
Epoch: 13, loss: tensor([[566.5469]], grad_fn=<PowBackward0>)
Epoch: 14, loss: tensor([[229.4752]], grad_fn=<PowBackward0>)
Epoch: 15, loss: tensor([[92.9793]], grad_fn=<PowBack

# Part C: Advanced Indexing and Broadcasting

In [78]:
data_df2 = pd.read_csv("../data/BTCUSDT.csv", sep='|', nrows=100, header=None, usecols=[1, 2, 3, 4, 5],
                       names=['open', 'high', 'low', 'close', 'volume'])
data_tensor2 = torch.tensor(data_df2.values, dtype=torch.float32)
print(data_tensor2[:5])

tensor([[4.2615e+03, 4.2615e+03, 4.2615e+03, 4.2615e+03, 1.7752e+00],
        [4.2615e+03, 4.2615e+03, 4.2615e+03, 4.2615e+03, 0.0000e+00],
        [4.2806e+03, 4.2806e+03, 4.2806e+03, 4.2806e+03, 2.6107e-01],
        [4.2615e+03, 4.2615e+03, 4.2615e+03, 4.2615e+03, 1.2008e-02],
        [4.2615e+03, 4.2615e+03, 4.2615e+03, 4.2615e+03, 1.4080e-01]])


In [89]:
high = data_tensor2[:, 1]
low = data_tensor2[:, 2]
close = data_tensor2[:, 3]

typical_price = (high + low + close) / 3
typical_price.shape

torch.Size([100])

In [90]:
volume = data_tensor2[:, -1]
min_vol = torch.min(volume)
vol_normalized = (volume - min_vol) / (torch.max(volume) - min_vol)
vol_normalized.shape

torch.Size([100])

In [99]:
bullish = data_tensor2[data_tensor2[:, 3] > data_tensor2[:, 0]]
bullish.shape

torch.Size([8, 5])